In [1]:
# Imports
import mtcnn
import cv2

In [2]:
# Read in test image
file_name = "./test_images/abba.png"
img = cv2.cvtColor( cv2.imread( file_name ), cv2.COLOR_BGR2RGB )

# Read in the weights for the MTCNN detector
detector = mtcnn.MTCNN( './data/mtcnn_weights.npy' )

In [3]:
# Detect faces in the image
faces = detector.detect_faces( img )

print( "Found {0} faces in {1}.".format( len(faces), file_name ) )

Found 4 faces in ./test_images/abba.png.


In [4]:
# Strong = 4
# Medium = 12
# Weak = >32
pixelateLevel = 4

pixelatedHeight, pixelatedWidth = ( pixelateLevel, pixelateLevel )

def pixelateROI( region ):
    '''
        Pixelate a region of an image by downsizing then resizing back to the original.
    '''
    height, width, _ = region.shape
    
    # Will resize the region of interest down to 32 x 32, or 12 x 12, etc etc
    resized = cv2.resize( region, ( pixelatedWidth, pixelatedHeight ), interpolation = cv2.INTER_LINEAR )
    
    # Will resize the 32 x 32 ROI back up to the original width and height
    return cv2.resize( resized, ( width, height ), interpolation = cv2.INTER_NEAREST )

In [5]:
def gaussianROI( region ):
    '''
        Blur a region of an image by Gaussian blur.
    '''
    height, width, _ = region.shape
    return cv2.GaussianBlur( region, ( 11, 11 ), 5 )

In [6]:
edge = 5
method = 'pixelate'
accepted = [ 'pixelate', 'gaussian' ]

# Privacy restrict regions of the image that contain faces.
for face in faces:
    x, y, w, h = face['box']
    
    if method in accepted:
        # Grab the R.O.I. of the image (the face)
        roi = img[ y - edge:y + h + edge, x - edge:x + w + edge ]
        modified_roi = None
        
        # Perform the blurring technique
        if method == 'pixelate':
            modified_roi = pixelateROI( roi )
        elif method == 'gaussian':
            modified_roi = gaussianROI( roi )
            
        # Replace the face with the modified, blurred R.O.I.
        img[ y - edge:y + h + edge, x - edge:x + w + edge ] = modified_roi
    else:
        print( "Unknown method: {0}".format(method) )

In [ ]:
# Show the image with blurred faces

cv2.imshow("Faces found", img)
cv2.waitKey(0)

In [9]:
video_capture = cv2.VideoCapture(0)

try:
    while True:

        ret, frame = video_capture.read()

        frame = cv2.resize(frame, (214, 160))

        rbgframe = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        faces = detector.detect_faces(rbgframe)

        for face in faces:
            x, y, w, h = face['box']
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

        cv2.imshow('Video', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
except:
    pass
finally:
    video_capture.release()
    cv2.destroyAllWindows()

In [11]:
video_capture = cv2.VideoCapture(0)

try:
    while True:

        ret, frame = video_capture.read()

        rgbframe = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        faces = detector.detect_faces(rgbframe)

        for face in faces:
            x, y, w, h = face['box']
            frame[y-5:y+h+5, x-5:x+w+5] = cv2.GaussianBlur(frame[y-5:y+h+5, x-5:x+w+5], (15, 15), 10)

        cv2.imshow('Video', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
except:
    pass
finally:
    video_capture.release()
    cv2.destroyAllWindows()

In [8]:
video_capture.release()
cv2.destroyAllWindows()

In [3]:
print("Found", len(faces), "faces!")

for face in faces:
    box = face['box']
    x, y, w, h = box
    cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 2)

Found 4 faces!
